# 🧠 Epileptic Seizure Detection using CNN
This notebook builds a CNN model to detect epileptic seizures from EEG time-series data.

In [ ]:
# 📦 Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
# 📥 Step 2: Load Dataset (local CSV path for Windows)
file_path = r"C:\\Users\\Shreetech\\Downloads\\Epileptic Seizure Recognition.csv"
df = pd.read_csv(file_path)

# Drop extra columns if present
df.drop(columns=["Unnamed", "Unnamed: 0"], inplace=True, errors='ignore')

# Convert to binary classification (1 = seizure, 0 = non-seizure)
df['y'] = df['y'].apply(lambda x: 1 if x == 1 else 0)

print("Shape:", df.shape)
df.head()

In [ ]:
# 📊 Step 3: Split Data
X = df.drop("y", axis=1).values
y = df["y"].values

# Normalize
X = X / np.max(X)

# Reshape for CNN (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 🧠 Step 4: Build CNN Model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 🏋️ Step 5: Train the Model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
# 📈 Step 6: Plot Accuracy
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# 📊 Step 7: Evaluate Model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))